# MAUNA LOA CO₂ FLASK DATA ANALYSIS
## A Beginner's Guide (Python Version)

This tutorial will walk you through how to work with real atmospheric CO₂ data collected at the **Mauna Loa Observatory** in Hawaii. This data is part of NOAA's Global Monitoring Laboratory (GML) long-term record of greenhouse gases. The Mauna Loa Observatory provides one of the longest continuous records of atmospheric CO₂, making it a vital source for atmospheric research worldwide.

Whether you're new to Python, atmospheric science, or data analysis, this guide is designed to help you get started step-by-step.

---

### In this tutorial, you'll learn how to:

- **Load data** in two common scientific formats:
  - Plain text (.txt)
  - NetCDF (.nc)

- **Explore the dataset structure**, including:
  - Viewing metadata
  - Listing available variables
  - Checking data size and data types

- **Filter the data**, such as:
  - Removing rejected values (flagged entries)
  - Narrowing down by date range or custom conditions

- **Visualize CO₂ levels** over time using Python plotting tools

- **Apply NOAA GML’s curve-fitting method** to:
  - Identify long-term trends
  - Separate out seasonal variations


## Step 1: Import Required Python Packages

To begin our analysis, we need to load some Python libraries. These packages will help us:
- Load and work with data tables
- Perform calculations and filter data
- Create time series plots

We’ll also import two custom Python modules provided by NOAA GML:
- `ccg_filter.py`: contains the filtering and curve-fitting tools
- `ccg_dates.py`: provides date conversion and handling functions used in GML’s analysis

**Make sure these two files (`ccg_filter.py` and `ccg_dates.py`) are in the same folder as this notebook.**
 

In [47]:
# Import common Python packages
import pandas as pd  # For loading and working with tables (dataframes)
import matplotlib.pyplot as plt  # For plotting
import matplotlib.dates as mdates  # For formatting time axes in plots
import datetime

# Import NOAA GML custom modules
import py.ccg_dates as ccg_dates # Date utilities
import py.ccg_filter as ccg_filter  # Data filtering and curve fitting

#### Tip: Set Default Plotting Styles Early

You can customize the default appearance of all your plots by setting `matplotlib` parameters at the beginning of your notebook. This isn’t required, but it helps keep your plots consistent and easier to read.

This is especially useful when you're making multiple plots throughout your analysis.

In [38]:
# Set default plot styles for consistent formatting
plt.rcParams['font.size'] = 16           # Base font size for text
plt.rcParams['axes.titlesize'] = 18      # Title font size for each plot
plt.rcParams['axes.labelsize'] = 16      # Font size for axis labels
plt.rcParams['xtick.labelsize'] = 14     # Font size for tick labels on x-axis
plt.rcParams['ytick.labelsize'] = 14     # Font size for tick labels on y-axis
plt.rcParams['legend.fontsize'] = 14     # Font size for plot legends

## Step 2: Load the CO₂ Data (Text Format)

NOAA's Global Monitoring Laboratory provides Mauna Loa CO₂ measurements in an ASCII text format. This file contains both metadata (information about the data) and the actual measurement values.
If you prefer to use the NetCDF file instead of the text file, see the Appendix for loading and processing instructions.

In this step, we'll:
- Locate and read the file
- Skip the metadata lines
- Load the data into a table (called a DataFrame) using the `pandas` library

To learn more about how these CO₂ flask samples are collected, visit:
https://gml.noaa.gov/ccgg/behind_the_scenes/surface.html 

In [39]:
# Insert the path to your downloaded ASCII text file
file_path = "./rawdata/co2_abp_surface-flask_1_ccgg_month.txt"

# First, find out how many header lines there are (they start with "#")
with open(file_path, "r") as file:
    header_lines = [line.strip() for line in file if line.startswith('#')]

# Count the number of header lines so we can skip them when reading the data
n = len(header_lines)

# Print the number of lines to skip
print(f"Number of header lines: {n}")

Number of header lines: 55


In [40]:
# Now read the data, skipping the header lines
flask_data = pd.read_csv(
    file_path,
    sep=r'\s+',       # Use whitespace as the separator
    skiprows=n,      # Skip the metadata lines
    names = ['site', 'year',  'month',  'value']         # First remaining line is the column header
)

# Display the first 5 rows of the data
flask_data.head()

,site,year,month,value
0,ABP,2006,10,380.91
1,ABP,2006,11,381.02
2,ABP,2006,12,381.12
3,ABP,2007,1,381.23
4,ABP,2007,2,381.29


## Step 3: Explore the Data

Now that we have loaded the CO₂ data, let’s explore the dataset.  


We will:
- Review metadata or file header info
- Examine column names and data types
- Check dataset size (rows × columns)
- Verify date/time formats


In [41]:
# Print the metadata/header rows at the top of the file
with open(file_path, "r") as file:
    header_lines = [line.strip() for line in file if line.startswith('#')]

print("File Header / Metadata:\n")
for line in header_lines:
    print(line)

File Header / Metadata:

# number_of_header_lines: 55
# comment:
# comment:  ************ USE OF GML DATA ****************
# comment:
# comment: These data are made freely available to the public and the scientific
# comment: community in the belief that their wide dissemination will lead to
# comment: greater understanding and new scientific insights. To ensure that GML
# comment: receives fair credit for their work please include relevant citation
# comment: text in publications. We encourage users to contact the data providers,
# comment: who can provide detailed information about the measurements and
# comment: scientific insight.  In cases where the data are central to a
# comment: publication, coauthorship for data providers may be appropriate.
# comment:
# comment:
# comment: Please see accompanying README file, if available, for citation text.
# comment:
# comment: Every effort is made to produce the most accurate and precise
# comment: measurements possible.  However, we reser

>The header metadata gives context about how the data was collected and what each column means - don't skip it!

In [42]:
# Display the column names in the DataFrame
flask_data.columns

Index(['site', 'year', 'month', 'value'], dtype='object')

In [43]:
# Check the number of rows and columns (shape of the DataFrame)
flask_data.shape

(34, 4)

In [44]:
# Check the data type of each column
flask_data.dtypes

site      object
year       int64
month      int64
value    float64
dtype: object

For easier plotting and analysis, we will convert the 'datetime' column values to a datetime object. 

In [45]:
# Convert the 'datetime' column to actual datetime objects, if it's not already
dates = []
for i in range(len(flask_data['site'])) :
    year = flask_data['year'][i]
    month = flask_data['month'][i]
    dates.append(datetime.datetime(year, month, 1))
flask_data['datetime'] = dates
flask_data['datetime'] = pd.to_datetime(flask_data['datetime'])

# Check data types again to confirm the change
flask_data.dtypes

site                object
year                 int64
month                int64
value              float64
datetime    datetime64[ns]
dtype: object

## Step 4: Filter the Data and Plot CO₂ Levels Over Time

Quality control (QC) flags are included in this dataset to help identify measurements that should be rejected for scientific use. 

### Quality Control (QC) Flags

The `qcflag` column uses a three-character code to describe measurement quality. Example: `"C.."`, `".S."`, or `"..."`.

- If the first character **is not** a dot (`.`), it means the measurement was **rejected** by the principal investigator.
- If the first character **is** a dot (`.`), the measurement is considered **valid**.

To ensure we're working with only high-quality data — and to keep our dataset manageable — we’ll:

- Remove any rows where the **first character in `qcflag` is not `.`** (indicating the measurement was rejected)
- Limit the dataset to a span of **15 years** (2008-2023)



In [46]:
# Filter to include only measurements from 2008-2023
flask_data = flask_data[(flask_data['year'] >= 2008) &
                        (flask_data['year'] <= 2023)]

# Create a mask for QC flags that start with '.' (valid data)
valid_qc_mask = flask_data['qcflag'].str.match(r'^\..*')

# Filter to include only valid data
good_flask_data = flask_data[valid_qc_mask].reset_index(drop=True)

# Display the first few rows of the cleaned data
good_flask_data.head()

KeyError: 'qcflag'

In [ ]:
# Check the new shape of the filtered dataset
# The number of columns will remain the same, but the amount of rows will decrease
good_flask_data.shape

The plot below shows **both valid and rejected** flask CO₂ measurements. 

Rejected data points often appear as outliers or noisy scatter, making it harder to see the real pattern in atmospheric CO₂ levels.

> *Each point represents one flask sample, plotted by sample date vs. CO₂ mole fraction.*


In [ ]:
# Plot showing both valid and rejected measurements
plt.figure(figsize=(25, 12))

# Create mask for valid data (qcflag starts with '.')
mask = flask_data['qcflag'].str.match(r'^\..*')

# Plot valid points
plt.scatter(
    flask_data.loc[mask, 'datetime'],
    flask_data.loc[mask, 'value'],
    s=25,
    label='Valid Data (.**)'  # QC flag starts with a dot
)

# Plot rejected points (first character is not a dot)
plt.scatter(
    flask_data.loc[~mask, 'datetime'],
    flask_data.loc[~mask, 'value'],
    s=25,
    color='red',
    label='Rejected Data (*..)'
)

# Format axes
plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO₂ at Mauna Loa Observatory: Full Dataset")
plt.xlabel('Year')
plt.ylabel('CO₂ mole fraction (ppm)')
plt.legend()
plt.show()

### Now Plot Only the Valid Data

Filtering out rejected and missing values reveals clearer patterns in the time series.

> *Note: The range of CO₂ mole fraction values decreases significantly - from a spread of ~300 ppm to a much cleaner ~50 ppm.*


In [ ]:
# Plot valid data only
plt.figure(figsize=(25, 12))

plt.scatter(good_flask_data['datetime'], good_flask_data['value'], s=25)

# Format axes
plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO₂ at Mauna Loa Observatory: Valid Data Only")
plt.xlabel('Year')
plt.ylabel('CO₂ mole fraction (ppm)')
plt.show()

#### Additional Data Exploration

To deepen our understanding, let's calculate some summary statistics on the filtered dataset:

In [ ]:
print("Summary statistics for CO₂ levels 2008 - 2023 (valid data only):")
print(good_flask_data['value'].describe())

We can also visualize the distribution of CO₂ measurements using a histogram: 

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(good_flask_data['value'], bins=30, color='skyblue', edgecolor='black')
plt.title("Distribution of CO$_2$ Mole Fraction (ppm)")
plt.xlabel("CO$_2$ (ppm)")
plt.ylabel("Frequency")
plt.show()

## Step 5: Apply NOAA GML's Curve Fitting Method

This method helps us to extract important patterns in the CO₂ data:  
- A long-term trend (overall rise in CO₂)  
- A yearly seasonal cycle (non-sinusoidal oscillations)  
- Short-term variations


This technique is based on the work by Thoning et al. (1989) and NOAA GML’s implementation. For more details and to follow along with the full tutorial, visit: https://gml.noaa.gov/ccgg/mbl/crvfit/crvfit.html



In [ ]:
# Set x and y data for the filter
xp = good_flask_data['time_decimal']
yp = good_flask_data['value']

# Create filter object using NOAA GML's curve fitting class
filt = ccg_filter.ccgFilter(
    xp,  # Time values in decimal years
    yp,  # CO2 mole fraction values
    shortterm=80,        # Short term smoothing cutoff in days
    longterm=667,        # Long term trend cutoff in days
    # Interval for evenly spaced data (0 means use original spacing)
    sampleinterval=0,
    numpolyterms=3,      # Polynomial terms (quadratic)
    numharmonics=4,      # Number of harmonics for seasonal cycle
    timezero=-1,         # Reference time zero for polynomial fitting
    gap=0,               # No gap-filling in interpolation
    use_gain_factor=False,  # Disable gain factor in harmonics
    debug=False          # Disable debug messages
)

We can now extract components of the fitted model for plotting:

- `x0`: equally spaced time points interpolated from xp  
- `y1`: function fit value at time x
- `y2`: polynomial fit value at time x
- `y3`: smoothed data value at time x
- `y4`: trend at time x

In [ ]:
# Extract components from the filter
x0 = filt.xinterp
y1 = filt.getFunctionValue(x0)
y2 = filt.getPolyValue(x0)
y3 = filt.getSmoothValue(x0)
y4 = filt.getTrendValue(x0)

# Convert decimal dates to datetime for plotting
x0_dates = [ccg_dates.datetimeFromDecimalDate(i) for i in x0]
xp_dates = [ccg_dates.datetimeFromDecimalDate(i) for i in xp]

We can plot various aspects of the `filt` object to visualize the different components: a long-term trend, a non-sinusoidal yearly cycle, and short term variations.

### Plot 1: Function Fit vs. Measured Data

In [ ]:
plt.figure(figsize=(25, 12))
plt.scatter(xp_dates, yp, s=30, alpha=0.9,
            edgecolors='black', label="Measured Data")
plt.plot(x0_dates, y1, color='orange', linewidth=3,
         alpha=0.9, label="Function Fit")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))  # Minor ticks every year
plt.gca().xaxis.set_major_locator(
    mdates.YearLocator(3))  # Major ticks every 3 years
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory")
plt.xlabel('Year')
plt.ylabel('CO$_2$ mole fraction (ppm)')
plt.legend()
plt.show()

### Plot 2: Polynomial and Function Fit vs. Measured Data

In [ ]:
plt.figure(figsize=(25, 12))
plt.scatter(xp_dates, yp, s=30, alpha=0.9,
            edgecolors='black', label="Measured Data")
plt.plot(x0_dates, y1, color='orange', linewidth=3,
         alpha=0.9, label="Function Fit")
plt.plot(x0_dates, y2, color='cyan', linewidth=3,
         alpha=0.9, label="Polynomial")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory")
plt.xlabel('Year')
plt.ylabel('CO$_2$ mole fraction (ppm)')
plt.legend()
plt.show()


### Plot 3: Residuals from Function Fit, Smoothed Residuals, and Trend Components

In [ ]:
# Residuals from function, smoothed residuals, and trend of residuals
resid_from_func = filt.resid
resid_smooth = filt.smooth
resid_trend = filt.trend

plt.figure(figsize=(25, 12))

plt.scatter(xp_dates, resid_from_func, s=30, edgecolors='black',
            label="Residuals From Function Fit")
plt.plot(x0_dates, resid_smooth, c='red', linewidth=3,
         alpha=0.8, label="Smoothed Residuals")
plt.plot(x0_dates, resid_trend, c='blue', linewidth=3,
         alpha=0.8, label="Trend of Residuals")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory")
plt.xlabel('Year')
plt.ylabel('Residual CO$_2$ (ppm)')
plt.legend()
plt.show()

At this stage, the curve fitting process is complete. Now, we combine different components of the fitted function and filtered data to isolate the specific signals we're interested in analyzing. The key components are:

- **Smoothed Data**: Represents the data with only short-term fluctuations removed. It combines the fitted function with residuals filtered using the short-term cutoff.
- **Trend**: Captures the long-term upward movement in CO₂ levels after removing seasonal cycles. This includes the polynomial portion of the fit plus residuals filtered with the long-term cutoff.
- **Detrended Seasonal Cycle**: Shows the yearly oscillation after removing the overall trend. This consists of the harmonic (seasonal) part of the fit combined with short-term filtered residuals.
- **Seasonal Amplitude**: Measures the size of the seasonal cycle, calculated as the difference between peak and trough in the detrended seasonal cycle.
- **Growth Rate**: The rate at which the long-term trend increases, found by taking the first derivative of the trend component.


## Plot 4: Measured Data with Smoothed and Trend Curves


In [ ]:
trend = filt.getTrendValue(xp)

plt.figure(figsize=(25, 12))

plt.scatter(xp_dates, yp, s=35, edgecolors='black', label="Measured Data")
plt.plot(xp_dates, trend, c='blue', linewidth=3,
         alpha=0.8, label="Trend Curve")
plt.plot(x0_dates, y3, c='red', linewidth=3, alpha=0.8, label="Smoothed Curve")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory")
plt.xlabel('Year')
plt.ylabel('CO$_2$ mole fraction (ppm)')
plt.legend()
plt.show()

### Plot 5: Smoothed Seasonal Cycle


In [ ]:
detrend = yp - trend
harmonics = filt.getHarmonicValue(x0)
smooth_cycle = harmonics + filt.smooth - filt.trend

plt.figure(figsize=(25, 12))

plt.scatter(xp_dates, detrend, s=40, edgecolors='black',
            label="Detrended Seasonal Cycle")
plt.plot(x0_dates, smooth_cycle, c='red', linewidth=3,
         alpha=0.8, label="Smoothed Curve")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory")
plt.xlabel('Year')
plt.ylabel('Detrended CO$_2$ mole fraction (ppm)')
plt.legend()
plt.show()

### Plot 6: Growth Rate of the Trend Curve

In [ ]:
growth_rate = filt.getGrowthRateValue(xp)

plt.figure(figsize=(25, 12))

plt.plot(xp_dates, growth_rate, c='green', linewidth=3,
         alpha=0.8, label="Trend Curve Growth Rate")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory")
plt.xlabel('Year')
plt.ylabel('CO$_2$ Growth Rate (ppm/year)')
plt.legend()
plt.show()

## Step 6: Monthly Means

Using monthly means computed by the filter, we can create a DataFrame and plot to visualize monthly variations.

In [ ]:
# Get monthly means (year, month, avg, sd, n)
mm = filt.getMonthlyMeans()

mm_df = pd.DataFrame(mm, columns=['year', 'month', 'avg', 'sd', 'n'])

# Convert year + month to datetime for plotting
mm_df['date'] = pd.to_datetime(mm_df[['year', 'month']].assign(day=1))

mm_df.head()

### Plot 7: Monthly Means with Measured Data


In [ ]:
plt.figure(figsize=(25, 12))

plt.scatter(xp_dates, yp, s=20, edgecolors='black', label="Measured Data")
plt.plot(mm_df['date'], mm_df['avg'], linestyle='-', marker='o', markersize=8,
         color='orange', markeredgecolor="black", label="Monthly Mean")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory")
plt.xlabel('Year')
plt.ylabel('CO$_2$ mole fraction (ppm)')
plt.legend()
plt.show()

## Summary and Takeaways

In this notebook, you learned how to work with real atmospheric CO₂ data from Mauna Loa Observatory. You practiced loading and exploring data, filtering out invalid measurements, and visualizing CO₂ levels over time. 

Then, you applied NOAA’s curve fitting method to separate long-term trends from seasonal and short-term variations in the data. This technique helps scientists better understand how CO₂ is changing in the atmosphere.

By completing these steps, you gained hands-on experience with data cleaning, time series analysis, and advanced curve fitting - all important skills for studying environmental data.

Keep exploring and experimenting with these tools to deepen your understanding of atmospheric science and data analysis!

## Appendix: Loading CO₂ Data (NetCDF Format)

In addition to the text file format, NOAA provides atmospheric CO₂ flask measurements in **NetCDF format** — a widely-used format for storing structured scientific data.

This section shows how to:

- Open the NetCDF file using the `xarray` package
- View metadata and available variables
- Convert selected variables into a `pandas.DataFrame`
- Use the resulting DataFrame with the rest of this tutorial

> *This is optional. The NetCDF and text files contain the same data. You only need to load one.*


In [ ]:
# *Note: You may need to install the package netcdf4, which is a dependency of xarray. This will likely not be included in your Anaconda Python environment.

# if necessary, uncomment the following line and run this cell to install
# pip install netcdf4

In [ ]:
# import required libraries
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
# Path to your NetCDF file
file_path = "co2_mlo_surface-flask_1_ccgg_event.nc"

# Open dataset
ds = xr.open_dataset(file_path)

# View dataset structure
ds

> The unique format of the NetCDF file allows the user to view both the metadata and dataset variables with only the command above - no need for extra print statements. 

We'll extract a subset of useful fields to create a structured DataFrame for analysis comparable to that performed above:

In [ ]:
# Variables to include in the DataFrame
variables_to_use = [
    'datetime', 'time_decimal', 'air_sample_container_id', 'value',
    'value_unc', 'latitude', 'longitude', 'altitude', 'elevation',
    'intake_height', 'method', 'event_number', 'instrument',
    'analysis_datetime', 'qcflag'
]

# Create the DataFrame
flask_data_nc = ds[variables_to_use].to_dataframe().reset_index()

# Decode all byte string columns in flask_data_nc
for col in flask_data_nc.columns:
    if flask_data_nc[col].dtype == object and isinstance(flask_data_nc[col].iloc[0], bytes):
        flask_data_nc[col] = flask_data_nc[col].str.decode('utf-8')


# Preview the data
flask_data_nc.head()

You can now use the `flask_data_nc` DataFrame (created from the NetCDF file) just like the text-based version. The structure is the same, so the rest of the analysis will work in the same way.


So, continue with filtering, plotting, and curve fitting just as described in steps 3-5 above:


### Step 3(b): Explore the NetCDF DataFrame

In [ ]:
# Column names
flask_data_nc.columns

In [ ]:
# Dimensions of the DataFrame: (rows, columns)
flask_data_nc.shape

In [ ]:
# Data types of each column
flask_data_nc.dtypes

In [ ]:
# Convert the 'datetime' column to datetime format (if it isn't already)
flask_data_nc['datetime'] = pd.to_datetime(flask_data_nc['datetime'])
flask_data_nc.dtypes

### Step 4(b): Filter and Visualize the CO₂ Data

We'll remove any rows that:

- Have a `qcflag` starting with any character other than `"."`
- Are not in the time range of 2008 - 2023

For simplicity, we will only include the plot of the filtered, valid CO₂ data points. The process of filtering to remove invalid or questionable data works the same way regardless of the data source, whether from the text file or the NetCDF file.


In [ ]:
# Filter to years 2008-2023
flask_data_nc = flask_data_nc[(flask_data_nc['datetime'].dt.year >= 2008) &
                              (flask_data_nc['datetime'].dt.year <= 2023)]

# Keep only entries where the first character of the qcflag is "."
good_nc_data = flask_data_nc[flask_data_nc['qcflag'].str.match(
    r'^\..*')].reset_index(drop=True)

good_nc_data.shape

In [ ]:
plt.figure(figsize=(25, 12))

plt.scatter(good_nc_data['datetime'], good_nc_data['value'], s=25)

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Atmospheric CO$_2$ at Mauna Loa Observatory: Valid Data Only (NetCDF)")
plt.xlabel('Year')
plt.ylabel('CO$_2$ mole fraction (ppm)')
plt.show()

### Step 5(b): Apply Curve Fitting to NetCDF Data
In this step, we begin the process of curve fitting and extracting the signal components from the CO₂ data.

Since the NetCDF data structure can vary slightly, we'll demonstrate the first few cells here. The rest of the analysis follows the same process as with the text-based dataset, so you can apply those steps directly once the initial data preparation is done.

In [ ]:
xp = good_nc_data['time_decimal']
yp = good_nc_data['value']

filt = ccg_filter.ccgFilter(xp, yp,
                            shortterm=80,
                            longterm=667,
                            sampleinterval=0,
                            numpolyterms=3,
                            numharmonics=4,
                            timezero=-1,
                            gap=0,
                            use_gain_factor=False,
                            debug=False)

In [ ]:
# Get time and curve components
x0 = filt.xinterp
y1 = filt.getFunctionValue(x0)
y2 = filt.getPolyValue(x0)
y3 = filt.getSmoothValue(x0)
y4 = filt.getTrendValue(x0)

# Convert to datetime for plotting
x0_dates = [ccg_dates.datetimeFromDecimalDate(i) for i in x0]
xp_dates = [ccg_dates.datetimeFromDecimalDate(i) for i in xp]

#### Plot Function Fit with Flask Data

In [ ]:
plt.figure(figsize=(25, 12))
plt.scatter(xp_dates, yp, s=30, edgecolors='black', label="Measured Data")
plt.plot(x0_dates, y1, color='orange', linewidth=3, label="Function Fit")

plt.gca().xaxis.set_minor_locator(mdates.YearLocator(1))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.grid(True, which='both', axis='x', linestyle='--',
         linewidth=0.7, color='lightgray')
plt.grid(True, which='major', axis='y', linestyle='--',
         linewidth=0.7, color='lightgray')

plt.title("Function Fit: Atmospheric CO₂ at Mauna Loa (NetCDF)")
plt.xlabel("Year")
plt.ylabel("CO₂ mole fraction (ppm)")
plt.legend()
plt.show()

This appendix demonstrated how to load and prepare atmospheric CO₂ data from a NetCDF file using the `xarray` and `pandas` libraries. While the core dataset is the same as the text version, some formatting steps, like converting byte strings, are unique to NetCDF.

Once processed, the NetCDF data can be used in exactly the same way as the text-based data for filtering, curve fitting, and visualization. This flexibility allows you to work with whichever format best suits your needs or workflow.
